In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, GaussianDropout, Flatten, SimpleRNN
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import Activation



In [82]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')
df.head(10)

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
5,1749-06,83.5
6,1749-07,94.8
7,1749-08,66.3
8,1749-09,75.9
9,1749-10,75.5


In [83]:
def get_train_test(url, split_percent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    print(df)
    data = np.array(df.values.astype('float32'))
    print(data)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

def create_NN(input_shape, activation):
    model = Sequential()
    model.add(Dense(
        64,
        input_shape=input_shape,
        activation=activation[0]
    ))
    model.add(Flatten())
    model.add(Dense(
        32,
        input_shape=input_shape,
        activation=activation[0]
    ))
    model.add(Flatten())
    model.add(Dense(
        1,
        input_shape=input_shape,
        activation=activation[0]
    ))
    model.add(Flatten())
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))   
    print(X) 
    return X, Y
 
time_steps = 12
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)


      Sunspots
0         58.0
1         62.6
2         70.0
3         55.7
4         85.0
...        ...
2815      71.8
2816      50.3
2817      55.8
2818      33.3
2819      33.4

[2820 rows x 1 columns]
[[58. ]
 [62.6]
 [70. ]
 ...
 [55.8]
 [33.3]
 [33.4]]
[[[0.22852638]
  [0.24665089]
  [0.2758077 ]
  ...
  [0.29747832]
  [0.6249015 ]
  [0.33569738]]

 [[0.28881007]
  [0.29905435]
  [0.3514578 ]
  ...
  [0.2588652 ]
  [0.24940896]
  [0.2970843 ]]

 [[0.2758077 ]
  [0.1713948 ]
  [0.17848699]
  ...
  [0.09141055]
  [0.11229314]
  [0.17336485]]

 ...

 [[0.04846336]
  [0.08747045]
  [0.03979511]
  ...
  [0.01182033]
  [0.00236407]
  [0.00118203]]

 [[0.01339637]
  [0.03073286]
  [0.01694247]
  ...
  [0.02245863]
  [0.03427896]
  [0.06067769]]

 [[0.07446808]
  [0.08077226]
  [0.09101655]
  ...
  [0.20961386]
  [0.25295505]
  [0.24231677]]]
[[[5.22064567e-01]
  [5.06304145e-01]
  [3.30575258e-01]
  [4.30654049e-01]
  [4.59810823e-01]
  [5.13396382e-01]
  [5.71709991e-01]
  [5.42553127e

In [84]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(
        hidden_units
        , input_shape=input_shape
        , activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [85]:
model = create_RNN(
    hidden_units=3
    , dense_units=1
    , input_shape=(time_steps,1)
    , activation=['tanh', 'tanh']
)
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

Epoch 1/20
187/187 - 1s - loss: 0.0560 - 1s/epoch - 7ms/step
Epoch 2/20
187/187 - 0s - loss: 0.0088 - 380ms/epoch - 2ms/step
Epoch 3/20
187/187 - 1s - loss: 0.0049 - 508ms/epoch - 3ms/step
Epoch 4/20
187/187 - 1s - loss: 0.0042 - 517ms/epoch - 3ms/step
Epoch 5/20
187/187 - 0s - loss: 0.0040 - 481ms/epoch - 3ms/step
Epoch 6/20
187/187 - 0s - loss: 0.0039 - 365ms/epoch - 2ms/step
Epoch 7/20
187/187 - 0s - loss: 0.0039 - 398ms/epoch - 2ms/step
Epoch 8/20
187/187 - 0s - loss: 0.0038 - 478ms/epoch - 3ms/step
Epoch 9/20
187/187 - 0s - loss: 0.0039 - 397ms/epoch - 2ms/step
Epoch 10/20
187/187 - 0s - loss: 0.0038 - 371ms/epoch - 2ms/step
Epoch 11/20
187/187 - 0s - loss: 0.0037 - 386ms/epoch - 2ms/step
Epoch 12/20
187/187 - 0s - loss: 0.0038 - 383ms/epoch - 2ms/step
Epoch 13/20
187/187 - 0s - loss: 0.0039 - 385ms/epoch - 2ms/step
Epoch 14/20
187/187 - 0s - loss: 0.0038 - 396ms/epoch - 2ms/step
Epoch 15/20
187/187 - 0s - loss: 0.0039 - 403ms/epoch - 2ms/step
Epoch 16/20
187/187 - 0s - loss: 0.00

In [86]:
import math
def print_error(trainY, testY, train_predict, test_predict):    
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    test_rmae = math.sqrt(mean_absolute_error(testY, test_predict))
    # Print RMSE
    print('Test RMSE: %.3f RMSE' % (test_rmse))    
    print('Test RMAE: %.3f RMSE' % (test_rmae))    
    print(r2_score(testY, test_predict))

# make predictions
train_predict = model.predict(trainX)
test_predict = model.predict(testX)
# Mean square error
print_error(trainY, testY, train_predict, test_predict)

2/2 [==============================] - 0s 7ms/step
Test RMSE: 0.087 RMSE
Test RMAE: 0.248 RMSE
0.8363902258313051


In [91]:
model = create_NN(
    input_shape=(time_steps,1)
    , activation=['relu', 'relu']
)
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

Epoch 1/20
187/187 - 1s - loss: 0.0095 - 1s/epoch - 6ms/step
Epoch 2/20
187/187 - 0s - loss: 0.0044 - 351ms/epoch - 2ms/step
Epoch 3/20
187/187 - 0s - loss: 0.0040 - 269ms/epoch - 1ms/step
Epoch 4/20
187/187 - 0s - loss: 0.0041 - 288ms/epoch - 2ms/step
Epoch 5/20
187/187 - 0s - loss: 0.0037 - 265ms/epoch - 1ms/step
Epoch 6/20
187/187 - 0s - loss: 0.0043 - 250ms/epoch - 1ms/step
Epoch 7/20
187/187 - 0s - loss: 0.0033 - 288ms/epoch - 2ms/step
Epoch 8/20
187/187 - 0s - loss: 0.0031 - 366ms/epoch - 2ms/step
Epoch 9/20
187/187 - 0s - loss: 0.0033 - 359ms/epoch - 2ms/step
Epoch 10/20
187/187 - 0s - loss: 0.0033 - 255ms/epoch - 1ms/step
Epoch 11/20
187/187 - 0s - loss: 0.0029 - 236ms/epoch - 1ms/step
Epoch 12/20
187/187 - 0s - loss: 0.0030 - 244ms/epoch - 1ms/step
Epoch 13/20
187/187 - 0s - loss: 0.0030 - 275ms/epoch - 1ms/step
Epoch 14/20
187/187 - 0s - loss: 0.0029 - 254ms/epoch - 1ms/step
Epoch 15/20
187/187 - 0s - loss: 0.0029 - 316ms/epoch - 2ms/step
Epoch 16/20
187/187 - 0s - loss: 0.00

In [90]:
def print_error(trainY, testY, train_predict, test_predict):    
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    test_rmae = math.sqrt(mean_absolute_error(testY, test_predict))
    # Print RMSE
    print('Test RMSE: %.3f RMSE' % (test_rmse))    
    print('Test RMAE: %.3f RMSE' % (test_rmae))    
    print(r2_score(testY, test_predict))

# make predictions
train_predict = model.predict(trainX)
test_predict = model.predict(testX)
# Mean square error
print_error(trainY, testY, train_predict, test_predict)

2/2 [==============================] - 0s 3ms/step
Test RMSE: 0.075 RMSE
Test RMAE: 0.236 RMSE
0.8790937326439998
